In [ ]:
!pip install openai

In [ ]:
!pip install sentence-transformers --upgrade


In [ ]:
!pip install ipywidgets jupyter --upgrade

In [ ]:
!pip install faiss-cpu sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import pickle

class VectorDBBuilder:
    def __init__(self, json_file, faiss_index_file="vector.index", metadata_file="metadata.pkl"):
        self.json_file = json_file
        self.index_file = faiss_index_file
        self.metadata_file = metadata_file
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.metadata = []

    def build_index(self):
        with open(self.json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        texts = []
        for entry in data:
            if entry.get("cleaned_content"):
                texts.append(entry["cleaned_content"])
                self.metadata.append({
                    "url": entry["url"],
                    "title": entry["title"]
                })

        embeddings = self.model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
        dimension = embeddings.shape[1]

        index = faiss.IndexFlatL2(dimension)
        index.add(np.array(embeddings))

        faiss.write_index(index, self.index_file)
        with open(self.metadata_file, 'wb') as f:
            pickle.dump(self.metadata, f)

        print(f"✅ FAISS index saved to {self.index_file}")
        print(f"✅ Metadata saved to {self.metadata_file}")

# Usage
if __name__ == "__main__":
    builder = VectorDBBuilder("drdo_scraped_with_pdfs2.json")
    builder.build_index()


In [ ]:
def search(query, top_k=5):
    import pickle
    import faiss
    from sentence_transformers import SentenceTransformer

    index = faiss.read_index("vector.index")
    with open("metadata.pkl", "rb") as f:
        metadata = pickle.load(f)

    model = SentenceTransformer('all-MiniLM-L6-v2')
    query_vector = model.encode([query])
    distances, indices = index.search(query_vector, top_k)

    results = []
    for i in indices[0]:
        results.append(metadata[i])

    return results

# Example usage
print(search("missile development heads"))

In [7]:
!pip install ctransformers sentence-transformers faiss-cpu

   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.4 MB 2.8 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/10.4 MB 2.2 MB/s eta 0:00:05
   ------ --------------------------------- 1.6/10.4 MB 2.1 MB/s eta 0:00:05
   ------- -------------------------------- 1.8/10.4 MB 2.1 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/10.4 MB 2.1 MB/s eta 0:00:04
   ---------- ----------------------------- 2.6/10.4 MB 2.0 MB/s eta 0:00:04
   ------------ --------------------------- 3.1/10.4 MB 2.0 MB/s eta 0:00:04
   ------------- -------------------------- 3.4/10.4 MB 1.9 MB/s eta 0:00:04
   --------------- ------------------------ 3.9/10.4 MB 1.9 MB/s eta 0:00:04
   ---------------- ----------------------- 4.2/10.4 MB 1.9 MB/s eta 0:00:04
   ------------------ --------------------- 4.7/10.4 MB 1.9 MB/s eta 0:00:03
   ----------

  You can safely remove it manually.


In [3]:
!pip uninstall numpy -y
!pip install numpy==1.26.4

Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Successfully uninstalled numpy-2.2.4


You can safely remove it manually.
You can safely remove it manually.


   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
    --------------------------------------- 0.3/15.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.8 MB 1.0 MB/s eta 0:00:15
   - -------------------------------------- 0.8/15.8 MB 1.1 MB/s eta 0:00:14
   -- ------------------------------------- 1.0/15.8 MB 1.1 MB/s eta 0:00:14
   --- ------------------------------------ 1.3/15.8 MB 1.0 MB/s eta 0:00:15
   --- ------------------------------------ 1.3/15.8 MB 1.0 MB/s eta 0:00:15
   --- ------------------------------------ 1.6/15.8 MB 976.0 kB/s eta 0:00:15
   ---- ----------------------------------- 1.8/15.8 MB 977.5 kB/s eta 0:00:15
   ----- ---------------------------------- 2.1/15.8 MB 1.0 MB/s eta 0:00:14
   ----- ---------------------------------- 2.1/15.8 MB 1.0 MB/s eta 0:00:14
   ----- ---------------------------------- 2.4/15.8 MB 938.7 kB/s eta 0:00:15
   ----- -----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddlepaddle 2.6.1 requires protobuf<=3.20.2,>=3.1.0; platform_system == "Windows", but you have protobuf 4.21.12 which is incompatible.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


In [5]:
pip uninstall transformers -y

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
os.environ["USE_TF"] = "0"

from ctransformers import AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss, pickle

# Load your FAISS vector DB + metadata
index = faiss.read_index("vector.index")
with open("metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

# Sentence embedding model (fast + free)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load tiny local LLM (.gguf)
llm = AutoModelForCausalLM.from_pretrained(
    "models",
    model_file="tinyllama.gguf",
    model_type="llama",
    gpu_layers=0,
    max_new_tokens=512,
    temperature=0.7
)

def retrieve_context(query, k=5):
    vec = embedding_model.encode([query])
    distances, indices = index.search(vec, k)
    return [metadata[i] for i in indices[0]]

def build_prompt(query, context):
    context_text = "\n\n".join(
        f"Title: {doc['title']}\nURL: {doc['url']}\n\n{doc.get('cleaned_content', '')[:1000]}"
        for doc in context
    )
    return f"""You are a helpful assistant. Use the context below to answer the question.

Context:
{context_text}

Question: {query}

Answer:"""

# Simple terminal agent
if __name__ == "__main__":
    print("🤖 DRDO Agent is ready. Type 'exit' to quit.")
    while True:
        q = input("\nYou: ")
        if q.lower() in ['exit', 'quit']:
            break
        docs = retrieve_context(q)
        prompt = build_prompt(q, docs)
        print("\nAgent:", llm(prompt))


RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Unable to convert function return value to a Python type! The signature was
	() -> handle